In [5]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
mu = 1.0  # Supongamos que mu es 1 para este ejemplo

# Tamaño del tablero, ej. 9x9 para un tablero estándar de Go pequeño
N = 19

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*N+j for i in range(N) for j in range(N)}

# Suponiendo que h_i(t) se puede calcular o se da como input. Aquí usamos un valor ficticio
# Por ejemplo, todos tienen 4 libertades inicialmente (completamente abierto)
h = {(i, j): 4 for i in range(N) for j in range(N)}

# Construir el Hamiltoniano Ising
h_values = {}
J_values = {}
for i in range(N):
    for j in range(N):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < N-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -1
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -1
        if j < N-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -1
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -1
        
        # Añadir el término de libertades
        h_values[spins[(i, j)]] = -mu * h[(i, j)]

# Crear el modelo binario
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)
# Muestrear el modelo
response = sampler.sample(model)

# Mostrar el inspector
ins.show(response)



Serving Inspector on http://127.0.0.1:18000/?problemId=2c183c83-04e3-41bb-a1f2-65ece14f934f

'http://127.0.0.1:18000/?problemId=2c183c83-04e3-41bb-a1f2-65ece14f934f'

CÓDIGO DE PARTIDA

In [54]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
mu = 1.0  # Supongamos que mu es 1 para este ejemplo

# Tamaño del tablero, ej. 9x9 para un tablero estándar de Go pequeño
N = 5

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*N+j for i in range(N) for j in range(N)}

# Configuración inicial del tablero
initial_board = {(i, j): 0 for i in range(N) for j in range(N)}
initial_board[(1, 1)] = 1  # Piedra blanca
initial_board[(4, 4)] = -1  # Piedra negra

# Configurar h_values de acuerdo con el estado inicial del tablero
h_values = {spins[(i, j)]: initial_board[(i, j)] for i in range(N) for j in range(N)}

# Suponiendo que h_i(t) se puede calcular o se da como input. Aquí usamos un valor ficticio
# Por ejemplo, todos tienen 4 libertades inicialmente (completamente abierto)
h = {(i, j): 4 for i in range(N) for j in range(N)}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(N):
    for j in range(N):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < N-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -1
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -1
        if j < N-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -1
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -1

# Crear el modelo binario
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)
# Muestrear el modelo
response = sampler.sample(model)

# Mostrar el inspector
ins.show(response)

Serving Inspector on http://127.0.0.1:18000/?problemId=32bb0729-d344-494a-980d-27aaab5573fa

'http://127.0.0.1:18000/?problemId=32bb0729-d344-494a-980d-27aaab5573fa'

In [55]:
# Crear una función para encontrar el próximo mejor movimiento
def find_best_move(initial_board, N, mu, sampler):
    # Genera una lista de todas las posiciones posibles para el próximo movimiento
    possible_positions = [(i, j) for i in range(N) for j in range(N) if initial_board[(i, j)] == 0]

    # Calcula la energía de cada posición posible si se coloca una piedra blanca en esa posición
    energies = []
    for position in possible_positions:
        board = initial_board.copy()
        board[position] = 1  # Coloca una piedra blanca
        energy = compute_local_energy(board, *position, mu, N)
        energies.append(energy)

    # Crea un modelo binario cuadrático para cada posición posible
    models = [dimod.BinaryQuadraticModel({spins[position]: energy}, {}, 0.0, dimod.SPIN) for position, energy in zip(possible_positions, energies)]

    # Utiliza D-Wave para encontrar la solución con la energía más baja para cada modelo
    responses = [sampler.sample(model) for model in models]

    # Extrae la energía de la mejor solución de cada respuesta
    energies = [response.first.energy for response in responses]
                      
    # Encuentra la posición con la energía más baja, que es el próximo mejor movimiento
    best_position = possible_positions[np.argmin(energies)]

    return best_position

# Llama a la función con el tablero inicial y otros parámetros
best_move = find_best_move(initial_board, N, mu, sampler)
print("El próximo mejor movimiento es en la posición:", (best_move[0] + 1, best_move[1] + 1))

El próximo mejor movimiento es en la posición: (1, 1)


In [49]:
import numpy as np

# Definir la función compute_local_energy
def compute_local_energy(board, i, j, mu, N):
    # Aquí debes implementar la lógica para calcular la energía local
    # Esta es solo una función de ejemplo, debes reemplazarla con tu propia lógica
    return mu * (i + j)

# Crear una lista de posiciones posibles para el próximo movimiento
possible_positions = [(i, j) for i in range(N) for j in range(N) if initial_board[(i, j)] == 0]

# Para cada posición posible, calcular la energía del tablero si se coloca una piedra blanca en esa posición
energies = []
for position in possible_positions:
    i, j = position
    board = initial_board.copy()
    board[(i, j)] = 1  # Colocar una piedra blanca en la posición
    energy = compute_local_energy(board, i, j, mu, N)
    energies.append(energy)

# Crear un modelo binario para cada posición posible
models = [dimod.BinaryQuadraticModel({spins[position]: energy}, {}, 0.0, dimod.SPIN) for position, energy in zip(possible_positions, energies)]

# Usar D-Wave para encontrar la posición con la energía más baja
responses = [sampler.sample(model) for model in models]
energies = [response.first.energy for response in responses]
best_position = possible_positions[np.argmin(energies)]

print("El próximo mejor movimiento es en la posición:", best_position)

El próximo mejor movimiento es en la posición: (4, 4)


In [36]:
# Importaciones necesarias
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
mu = 1.0
N = 5

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*N+j for i in range(N) for j in range(N)}

# Configuración inicial del tablero
initial_board1 = {(i, j): 0 for i in range(N) for j in range(N)}
initial_board1[(1, 2)] = 1  # Piedra blanca
initial_board1[(3, 1)] = -1  # Piedra negra

initial_board2 = np.zeros((N, N))
initial_board2[(2, 2)] = 1  # Piedra blanca
initial_board2[(3, 3)] = -1  # Piedra negra

# Funciones para calcular la energía y las libertades
def sigmoid_transformed(x):
    return 1 / (1 + np.exp(-1 / np.sqrt(x + 0.0001)))

def get_neighbors_positions(i, j, BOARD_SIZE):
    return [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

def compute_liberties(board, i, j, BOARD_SIZE):
    empty_spots = [1 if board[k][l] == 0 else 0 for k, l in get_neighbors_positions(i, j, BOARD_SIZE) if k >= 0 and l >= 0 and k < BOARD_SIZE and l < BOARD_SIZE]
    return sum(empty_spots)

def compute_local_energy(board, i, j, mu, BOARD_SIZE):
    xi = board[i][j]
    neighbors = [board[x][y] for x, y in get_neighbors_positions(i, j, BOARD_SIZE) if x >= 0 and y >= 0 and x < BOARD_SIZE and y < BOARD_SIZE]
    liberties = compute_liberties(board, i, j, BOARD_SIZE)
    sigmoid_liberties_effect = sigmoid_transformed(liberties)
    return xi * sum(neighbors) - mu * sigmoid_liberties_effect

# Configurar h_values de acuerdo con el estado inicial del tablero
h_values1 = {spins[(i, j)]: initial_board1[(i, j)] for i in range(N) for j in range(N)}
h_values2 = {spins[(i, j)]: compute_local_energy(initial_board2, i, j, mu, N) for i in range(N) for j in range(N)}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(N):
    for j in range(N):
        if i < N-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -1
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -1
        if j < N-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -1
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -1

# Crear los modelos binarios
model1 = dimod.BinaryQuadraticModel(h_values1, J_values, 0.0, dimod.SPIN)
model2 = dimod.BinaryQuadraticModel(h_values2, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)

# Muestrear los modelos
response1 = sampler.sample(model1)
response2 = sampler.sample(model2)

# Mostrar las respuestas
print("Respuesta del modelo 1:")
print(response1)
print("\nRespuesta del modelo 2:")
print(response2)

# Crear una lista de posiciones posibles para el próximo movimiento
possible_positions = [(i, j) for i in range(N) for j in range(N) if initial_board2[i][j] == 0]

# Para cada posición posible, calcular la energía del tablero si se coloca una piedra blanca en esa posición
energies = []
for position in possible_positions:
    i, j = position
    board = initial_board2.copy()
    board[i][j] = 1  # Colocar una piedra blanca en la posición
    energy = compute_local_energy(board, i, j, mu, N)
    energies.append(energy)

# Crear un modelo binario para cada posición posible
models = [dimod.BinaryQuadraticModel({spins[position]: energy}, {}, 0.0, dimod.SPIN) for position, energy in zip(possible_positions, energies)]

# Usar D-Wave para encontrar la posición con la energía más baja
responses = [sampler.sample(model) for model in models]
energies = [response.first.energy for response in responses]
best_position = possible_positions[np.argmin(energies)]

print("El próximo mejor movimiento es en la posición:", best_position)

Respuesta del modelo 1:
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 24 energy num_oc. ...
0 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 ... +1  -80.0       1 ...
['SPIN', 1 rows, 1 samples, 25 variables]

Respuesta del modelo 2:
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 ... 24     energy num_oc. ...
0 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 ... +1 -96.155814       1 ...
['SPIN', 1 rows, 1 samples, 25 variables]
El próximo mejor movimiento es en la posición: (3, 4)


In [37]:
# Importaciones necesarias
# ...

# Configuración del solver
# ...

# Parámetros del modelo
# ...

# Crear variables de Spin para cada posición en el tablero
# ...

# Configuración inicial del tablero
# ...

# Funciones para calcular la energía y las libertades
# ...

# Configurar h_values de acuerdo con el estado inicial del tablero
# ...

# Construir el Hamiltoniano Ising
# ...

# Crear los modelos binarios
# ...

# Crear un sampler
# ...

# Muestrear los modelos
# ...

# Mostrar las respuestas
# ...

# Crear una lista de posiciones posibles para el próximo movimiento
# ...

# Para cada posición posible, calcular la energía del tablero si se coloca una piedra blanca en esa posición
# ...

# Crear un modelo binario para cada posición posible
# ...

# Usar D-Wave para encontrar la posición con la energía más baja
responses = [sampler.sample(model) for model in models]
energies = [response.first.energy for response in responses]

# Imprimir todas las posibles posiciones que la QPU puede calcular
for position, energy, response in zip(possible_positions, energies, responses):
    print(f"Posición: {position}, Energía: {energy}, Respuesta: {response}")

best_position = possible_positions[np.argmin(energies)]

print("El próximo mejor movimiento es en la posición:", best_position)

Posición: (0, 0), Energía: -0.669757639496628, Respuesta:    0    energy num_oc. chain_.
0 +1 -0.669758       1     0.0
['SPIN', 1 rows, 1 samples, 1 variables]
Posición: (0, 1), Energía: -0.6404552599425836, Respuesta:    1    energy num_oc. chain_.
0 +1 -0.640455       1     0.0
['SPIN', 1 rows, 1 samples, 1 variables]
Posición: (0, 2), Energía: -0.6404552599425836, Respuesta:    2    energy num_oc. chain_.
0 +1 -0.640455       1     0.0
['SPIN', 1 rows, 1 samples, 1 variables]
Posición: (0, 3), Energía: -0.6404552599425836, Respuesta:    3    energy num_oc. chain_.
0 +1 -0.640455       1     0.0
['SPIN', 1 rows, 1 samples, 1 variables]
Posición: (0, 4), Energía: -0.669757639496628, Respuesta:    4    energy num_oc. chain_.
0 +1 -0.669758       1     0.0
['SPIN', 1 rows, 1 samples, 1 variables]
Posición: (1, 0), Energía: -0.6404552599425836, Respuesta:    5    energy num_oc. chain_.
0 +1 -0.640455       1     0.0
['SPIN', 1 rows, 1 samples, 1 variables]
Posición: (1, 1), Energía: -0.

In [34]:
# Resto del código...

# Imprimir todas las posibles posiciones que la QPU puede calcular
all_responses = []
for position, energy, response in zip(possible_positions, energies, responses):
    all_responses.append((position, energy, response))
    print(f"Posición: {position}, Energía: {energy}, Respuesta: {response}")

# Buscar la respuesta para la posición específica
specific_position = (3, 4)
specific_response = next((resp for resp in all_responses if resp[0] == specific_position), None)

if specific_response is not None:
    print(f"Respuesta para la posición {specific_position}: {specific_response}")
else:
    print(f"No se encontró una respuesta para la posición {specific_position}"
best_position = possible_positions[np.argmin(energies)]

print("El próximo mejor movimiento es en la posición:", best_position)

SyntaxError: invalid syntax (1322767222.py, line 17)

In [13]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
mu = 1.0  # Supongamos que mu es 1 para este ejemplo

# Tamaño del tablero, ej. 9x9 para un tablero estándar de Go pequeño
N = 5

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*N+j for i in range(N) for j in range(N)}

# Configuración inicial del tablero
initial_board = np.zeros((N, N))
initial_board[(2, 2)] = 1  # Piedra blanca
initial_board[(3, 3)] = -1  # Piedra negra

# Funciones del primer script adaptadas para calcular la energía y las libertades
def sigmoid_transformed(x):
    return 1 / (1 + np.exp(-1 / np.sqrt(x + 0.0001)))

def get_neighbors_positions(i, j, BOARD_SIZE):
    return [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

def compute_liberties(board, i, j, BOARD_SIZE):
    empty_spots = [1 if board[k][l] == 0 else 0 for k, l in get_neighbors_positions(i, j, BOARD_SIZE) if k >= 0 and l >= 0 and k < BOARD_SIZE and l < BOARD_SIZE]
    return sum(empty_spots)

def compute_local_energy(board, i, j, mu, BOARD_SIZE):
    xi = board[i][j]
    neighbors = [board[x][y] for x, y in get_neighbors_positions(i, j, BOARD_SIZE) if x >= 0 and y >= 0 and x < BOARD_SIZE and y < BOARD_SIZE]
    liberties = compute_liberties(board, i, j, BOARD_SIZE)
    sigmoid_liberties_effect = sigmoid_transformed(liberties)
    return xi * sum(neighbors) - mu * sigmoid_liberties_effect

# Configurar h_values de acuerdo con el estado inicial del tablero y la energía local
h_values = {spins[(i, j)]: compute_local_energy(initial_board, i, j, mu, N) for i in range(N) for j in range(N)}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(N):
    for j in range(N):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < N-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -1
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -1
        if j < N-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -1
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -1

# Crear el modelo binario
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)
# Muestrear el modelo
response = sampler.sample(model)

# Mostrar el inspector
ins.show(response)

Serving Inspector on http://127.0.0.1:18000/?problemId=6b039e69-a3e4-488d-b4b5-cdfc618e59c4

'http://127.0.0.1:18000/?problemId=6b039e69-a3e4-488d-b4b5-cdfc618e59c4'

In [21]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
mu = 1.0  # Supongamos que mu es 1 para este ejemplo

# Tamaño del tablero, ej. 9x9 para un tablero estándar de Go pequeño
N = 5

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*N+j for i in range(N) for j in range(N)}

# Configuración inicial del tablero
initial_board = np.zeros((N, N))
initial_board[(2, 2)] = 1  # Piedra blanca
initial_board[(3, 3)] = -1  # Piedra negra

# Funciones del primer script adaptadas para calcular la energía y las libertades
def sigmoid_transformed(x):
    return 1 / (1 + np.exp(-1 / np.sqrt(x + 0.0001)))

def get_neighbors_positions(i, j, BOARD_SIZE):
    return [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

def compute_liberties(board, i, j, BOARD_SIZE):
    empty_spots = [1 if board[k][l] == 0 else 0 for k, l in get_neighbors_positions(i, j, BOARD_SIZE) if k >= 0 and l >= 0 and k < BOARD_SIZE and l < BOARD_SIZE]
    return sum(empty_spots)

def compute_local_energy(board, i, j, mu, BOARD_SIZE):
    xi = board[i][j]
    neighbors = [board[x][y] for x, y in get_neighbors_positions(i, j, BOARD_SIZE) if x >= 0 and y >= 0 and x < BOARD_SIZE and y < BOARD_SIZE]
    liberties = compute_liberties(board, i, j, BOARD_SIZE)
    sigmoid_liberties_effect = sigmoid_transformed(liberties)
    return xi * sum(neighbors) - mu * sigmoid_liberties_effect

# Configurar h_values de acuerdo con el estado inicial del tablero y la energía local
h_values = {spins[(i, j)]: compute_local_energy(initial_board, i, j, mu, N) for i in range(N) for j in range(N)}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(N):
    for j in range(N):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < N-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -1
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -1
        if j < N-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -1
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -1

# Crear el modelo binario
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)

# Muestrear el modelo
response = sampler.sample(model)

# Función para imprimir todas las respuestas
def print_all_responses(response):
    for sample, energy in response.data(['sample', 'energy']):
        print(f"Configuración: {sample}, Energía: {energy}")

print_all_responses(response)

# Función para cambiar el tamaño del tablero
def change_board_size(new_size):
    global N
    N = new_size
    global initial_board
    initial_board = np.zeros((N, N))
    global spins
    spins = {(i, j): i*N+j for i in range(N) for j in range(N)}
    global h_values
    h_values = {spins[(i, j)]: compute_local_energy(initial_board, i, j, mu, N) for i in range(N) for j in range(N)}

# Cambiar el tamaño del tablero a 9
change_board_size(9)

# Mostrar el inspector
ins.show(response)

Configuración: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1}, Energía: -96.15581354755936


Serving Inspector on http://127.0.0.1:18000/?problemId=a4aa2aa8-7e19-4b27-96ae-70258ddb1bcd

'http://127.0.0.1:18000/?problemId=a4aa2aa8-7e19-4b27-96ae-70258ddb1bcd'

In [16]:
# Importaciones necesarias
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
mu = 1.0
N = 5

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*N+j for i in range(N) for j in range(N)}

# Configuración inicial del tablero
initial_board1 = {(i, j): 0 for i in range(N) for j in range(N)}
initial_board1[(2, 2)] = 1  # Piedra blanca
initial_board1[(3, 3)] = -1  # Piedra negra

initial_board2 = np.zeros((N, N))
initial_board2[(2, 2)] = 1  # Piedra blanca
initial_board2[(3, 3)] = -1  # Piedra negra

# Funciones para calcular la energía y las libertades
def sigmoid_transformed(x):
    return 1 / (1 + np.exp(-1 / np.sqrt(x + 0.0001)))

def get_neighbors_positions(i, j, BOARD_SIZE):
    return [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

def compute_liberties(board, i, j, BOARD_SIZE):
    empty_spots = [1 if board[k][l] == 0 else 0 for k, l in get_neighbors_positions(i, j, BOARD_SIZE) if k >= 0 and l >= 0 and k < BOARD_SIZE and l < BOARD_SIZE]
    return sum(empty_spots)

def compute_local_energy(board, i, j, mu, BOARD_SIZE):
    xi = board[i][j]
    neighbors = [board[x][y] for x, y in get_neighbors_positions(i, j, BOARD_SIZE) if x >= 0 and y >= 0 and x < BOARD_SIZE and y < BOARD_SIZE]
    liberties = compute_liberties(board, i, j, BOARD_SIZE)
    sigmoid_liberties_effect = sigmoid_transformed(liberties)
    return xi * sum(neighbors) - mu * sigmoid_liberties_effect

# Configurar h_values de acuerdo con el estado inicial del tablero
h_values1 = {spins[(i, j)]: initial_board1[(i, j)] for i in range(N) for j in range(N)}
h_values2 = {spins[(i, j)]: compute_local_energy(initial_board2, i, j, mu, N) for i in range(N) for j in range(N)}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(N):
    for j in range(N):
        if i < N-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -1
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -1
        if j < N-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -1
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -1

# Crear los modelos binarios
model1 = dimod.BinaryQuadraticModel(h_values1, J_values, 0.0, dimod.SPIN)
model2 = dimod.BinaryQuadraticModel(h_values2, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)

# Muestrear los modelos
response1 = sampler.sample(model1)
response2 = sampler.sample(model2)

# Mostrar las respuestas
print("Respuesta del modelo 1:")
print(response1)
print("\nRespuesta del modelo 2:")
print(response2)

Respuesta del modelo 1:
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 24 energy num_oc. ...
0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ... -1  -80.0       1 ...
['SPIN', 1 rows, 1 samples, 25 variables]

Respuesta del modelo 2:
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 ... 24     energy num_oc. ...
0 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 ... +1 -96.155814       1 ...
['SPIN', 1 rows, 1 samples, 25 variables]


OIsing con muchos parámetros sin correlacionar

In [11]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
params = {
    'J': np.random.uniform(0.1, 1.0),  # Interacción entre piedras adyacentes.
    'J_ij': np.random.uniform(0.1, 1.0),  # Interacción a larga distancia.
    'alpha': np.random.uniform(0.1, 1.0),  # Coeficiente de decaimiento para la interacción a larga distancia.
    'L': np.random.uniform(0.1, 1.0),  # Importancia de las situaciones de Ko y Seki.
    'M': np.random.uniform(0.1, 1.0),  # Importancia de las condiciones de borde.
    'R': np.random.uniform(0.1, 1.0),  # Ajuste basado en la fase del juego.
    'size': 9,  # Tamaño del tablero.
}

# Definir el estado inicial del tablero para un tablero de 9x9.
board = np.random.choice([-1, 0, 1], size=(params['size'], params['size']))

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*params['size']+j for i in range(params['size']) for j in range(params['size'])}

# Configurar h_values de acuerdo con el estado inicial del tablero
h_values = {spins[(i, j)]: board[i, j] for i in range(params['size']) for j in range(params['size'])}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(params['size']):
    for j in range(params['size']):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < params['size']-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -params['J']
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -params['J']
        if j < params['size']-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -params['J']
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -params['J']

# Crear el modelo binario
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)
# Muestrear el modelo
response = sampler.sample(model)

# Mostrar el inspector
ins.show(response)

Serving Inspector on http://127.0.0.1:18000/?problemId=82b8d85e-ff20-40ba-aca5-c4e06d2b3d87

'http://127.0.0.1:18000/?problemId=82b8d85e-ff20-40ba-aca5-c4e06d2b3d87'

ISING con parámetros correlacionados

In [12]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.inspector import show
import dimod
import dwave.inspector as ins

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'

# Tu token de D-Wave
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"

# Inicialización del solver con la configuración especificada
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros base del modelo
base_J = 0.5  # Interacción base entre piedras adyacentes.
base_alpha = 0.8  # Coeficiente base de decaimiento para la interacción a larga distancia.
base_L = 0.3  # Importancia base de las situaciones de Ko y Seki.
base_M = 0.7  # Importancia base de las condiciones de borde.
base_R = 0.6  # Ajuste base basado en la fase del juego.

# Tamaño del tablero
size = 9

# Correlación entre parámetros
J = base_J * np.random.uniform(0.9, 1.1)
alpha = base_alpha * np.random.uniform(0.9, 1.1)
L = base_L * np.random.uniform(0.9, 1.1)
M = base_M * J  # M está directamente correlacionado con J.
R = base_R * alpha  # R está directamente correlacionado con alpha.

# Definir el estado inicial del tablero para un tablero de 9x9.
board = np.random.choice([-1, 0, 1], size=(size, size))

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): i*size+j for i in range(size) for j in range(size)}

# Configurar h_values de acuerdo con el estado inicial del tablero
h_values = {spins[(i, j)]: board[i, j] for i in range(size) for j in range(size)}

# Construir el Hamiltoniano Ising
J_values = {}
for i in range(size):
    for j in range(size):
        # Añade la interacción con los vecinos si están dentro de los límites del tablero
        if i < size-1:  # Sur
            J_values[(spins[(i, j)], spins[(i+1, j)])] = -J
        if i > 0:  # Norte
            J_values[(spins[(i, j)], spins[(i-1, j)])] = -J
        if j < size-1:  # Este
            J_values[(spins[(i, j)], spins[(i, j+1)])] = -J
        if j > 0:  # Oeste
            J_values[(spins[(i, j)], spins[(i, j-1)])] = -J

# Crear el modelo binario cuadrático
model = dimod.BinaryQuadraticModel(h_values, J_values, 0.0, dimod.SPIN)

# Crear un sampler
sampler = EmbeddingComposite(solver)
# Muestrear el modelo
response = sampler.sample(model)

# Mostrar el inspector
ins.show(response)

Serving Inspector on http://127.0.0.1:18000/?problemId=ed836c2d-389b-4de1-93e6-df007b01b872

'http://127.0.0.1:18000/?problemId=ed836c2d-389b-4de1-93e6-df007b01b872'

In [22]:
# Crear una lista de posiciones posibles para el próximo movimiento
possible_positions = [(i, j) for i in range(N) for j in range(N) if initial_board2[i][j] == 0]

# Para cada posición posible, calcular la energía del tablero si se coloca una piedra blanca en esa posición
energies = []
for position in possible_positions:
    i, j = position
    board = initial_board2.copy()
    board[i][j] = 1  # Colocar una piedra blanca en la posición
    energy = compute_local_energy(board, i, j, mu, N)
    energies.append(energy)

# Crear un modelo binario para cada posición posible
models = [dimod.BinaryQuadraticModel({spins[position]: energy}, {}, 0.0, dimod.SPIN) for position, energy in zip(possible_positions, energies)]

# Usar D-Wave para encontrar la posición con la energía más baja
responses = [sampler.sample(model) for model in models]
energies = [response.first.energy for response in responses]
best_position = possible_positions[np.argmin(energies)]

print("El próximo mejor movimiento es en la posición:", best_position)

IndexError: index 5 is out of bounds for axis 0 with size 5

In [23]:
# Usar D-Wave para encontrar la posición con la energía más baja
responses = [sampler.sample(model) for model in models]

# Imprimir todas las respuestas
for i, response in enumerate(responses):
    print(f"Posición: {possible_positions[i]}")
    for sample, energy in response.data(['sample', 'energy']):
        print(f"  Configuración: {sample}, Energía: {energy}")

Posición: (0, 0)
  Configuración: {0: 1}, Energía: -0.669757639496628
Posición: (0, 1)
  Configuración: {1: 1}, Energía: -0.6404552599425836
Posición: (0, 2)
  Configuración: {2: 1}, Energía: -0.6404552599425836
Posición: (0, 3)
  Configuración: {3: 1}, Energía: -0.6404552599425836
Posición: (0, 4)
  Configuración: {4: 1}, Energía: -0.669757639496628
Posición: (1, 0)
  Configuración: {5: 1}, Energía: -0.6404552599425836
Posición: (1, 1)
  Configuración: {6: 1}, Energía: -0.6224578624550682
Posición: (1, 2)
  Configuración: {7: -1}, Energía: -0.35954474005741643
Posición: (1, 3)
  Configuración: {8: 1}, Energía: -0.6224578624550682
Posición: (1, 4)
  Configuración: {9: 1}, Energía: -0.6404552599425836
Posición: (2, 0)
  Configuración: {10: 1}, Energía: -0.6404552599425836
Posición: (2, 1)
  Configuración: {11: -1}, Energía: -0.35954474005741643
Posición: (2, 3)
  Configuración: {13: 1}, Energía: -0.669757639496628
Posición: (2, 4)
  Configuración: {14: 1}, Energía: -0.6404552599425836
P